In [ ]:
# coding=utf-8

from datetime import datetime, timedelta
import pandas as pd
import pandahouse as ph
from io import StringIO
import requests
from datetime import date
import telegram 
import matplotlib.pyplot as plt
import numpy as np
import io
import seaborn as sns

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context

#параметры соединения 
connection = {'host': 'https://clickhouse.lab.karpov.courses',
                  'database':'simulator_20231113',
                   'user':'student',
                   'password':'dpo_python_2020'
}

# Дефолтные параметры, которые прокидываются в таски
default_args = {
    'owner': 'd-kulikova', # Владелец операции 
    'depends_on_past': False, # Зависимость от прошлых запусков
    'retries': 2, # Кол-во попыток выполнить DAG
    'retry_delay': timedelta(minutes=5), # Промежуток между перезапусками
    'start_date': datetime(2023, 10, 1), # Дата начала выполнения DAG
}

# Интервал запуска DAG
schedule_interval = '0 11 * * *' 

def bot_kulikova_report(chat=None):
    chat_id = chat or ***#(удален)
    
    my_token = '***'#(удален)
    bot = telegram.Bot(token=my_token)
    

    query = """ SELECT count(distinct user_id) as dau, 
                       countIf(action = 'view') as views, 
                       countIf(action = 'like') as likes,
                       round(likes/views, 3) as ctr
                FROM {db}.feed_actions
                WHERE toDate(time)=yesterday() """
    
    df_day_ago = ph.read_clickhouse(query, connection=connection)
    
    
    query = """SELECT toDate(time) as day,
                      count(distinct user_id) as dau, 
                      countIf(action = 'view') as views, 
                      countIf(action = 'like') as likes,
                      round(likes/views, 3) as ctr
                  
                FROM {db}.feed_actions
                WHERE toDate(time) BETWEEN DATE_SUB(NOW(), INTERVAL 7 DAY) AND NOW()
                GROUP BY day """
    
    df_week = ph.read_clickhouse(query, connection=connection)
    
    # задаем параметры времени
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    sevendays = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
    week_before = pd.date_range(start=sevendays, end=yesterday) 
    week_before = tuple([i.strftime("%Y-%m-%d") for i in week_before])

    # считаем метрики за вчерашний день
    dau_day = df_day_ago.loc[0, 'dau']
    views_day = df_day_ago.loc[0, 'views']
    likes_day = df_day_ago.loc[0, 'likes']
    ctr_day = df_day_ago.loc[0, 'ctr']
    
    # отправляем метрики в чат 
    msg = f'''Cтатистика за {yesterday}: DAU – {dau_day} пользователей, Просмотры – {views_day}, Лайки – {likes_day}, CTR -{ctr_day}'''
    
    # для графика лайков и просмотров
    actions_week = pd.melt(df_week, id_vars=['day'], value_vars=['views', 'likes'], var_name='action')

    # график для недельной статистики
    sns.set_theme(style='white')
    fig, axs = plt.subplots(ncols=3, figsize=(28, 8))
    fig.suptitle(f'Статистика за неделю c {week_before[0]} по {week_before[-1]}', fontsize=16)
    sns.lineplot(data=df_week, x='day', y='dau', ax=axs[0])
    axs[0].set_title('DAU')
    
    sns.lineplot(data = actions_week, x='day', y='value', hue='action', markers=True, ax=axs[1])
    axs[1].set_title('Likes and Views')
    
    sns.lineplot(data=df_week, x='day', y='ctr', ax=axs[2])
    axs[2].set_title('CTR')


@dag(default_args=default_args, catchup=False, schedule_interval=schedule_interval)
def dag_kulikova_bot_feed_report():
    
    @task
    def make_report():
        bot_kulikova_report(chat=***#(удален))
        
    make_report()

dag_kulikova_bot_feed_report = dag_kulikova_bot_feed_report()
